### Data collection to label the offshores wind-parcs of Germany on the map

In [173]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [154]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the page
link = "https://de.wikipedia.org/wiki/Liste_der_deutschen_Offshore-Windparks"

# Fetch the page content
response = requests.get(link)

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the first table on the page (or specify the exact table if needed)
table = soup.find('table', {'class': 'wikitable'})

# Extract all rows from the table
rows = table.find_all('tr')

# Create a list to hold the rows of data
data = []

# Extract the header (columns) from the first row
headers = [header.get_text(strip=True) for header in rows[0].find_all('th')]

# Extract the table rows (data)
for row in rows[1:]:
    columns = row.find_all('td')
    if columns:  # Only process rows with data
        data.append([column.get_text(strip=True) for column in columns])

# Convert to a DataFrame
offshores = pd.DataFrame(data, columns=headers)

# Ensure the 'Leistung' column is treated as a string to retain the comma
#df['Leistung (MW)'] = df['Leistung (MW)'].astype(str)

# Now you should see the 'Leistung' column with commas intact
#print(df['Leistung (MW)'])
offshores

,Name,Leistung(MW),"Windkraftanlagentyp(Leistung, Rotordurchmesser)",AnzahlWKAs,Koordinaten,Status,Inbetrieb­nahme(Jahr),"Quellen, Anmerkungen"
0,Nordsee,None,None,None,None,None,None,None
1,alpha ventus,"60,48","6 × REpower 5M(5,08 MW, 126,5 m)6 ×AREVAM5000-...",12,"54° 0′ 30″ N, 6° 35′ 54″ O",in Betrieb,2010,
2,BARD Offshore I,400,"BARD5.0(5,0 MW, 122,0 m)",80,"54° 21′ 30″ N, 5° 58′ 30″ O",in Betrieb,2013,
3,Riffgat,"113,4","Siemens SWT-3.6-120(3,78 MW, 120,0 m)",30,"53° 41′ 24″ N, 6° 28′ 48″ O",in Betrieb,2014,[5]
4,Amrumbank West,"302,4","Siemens SWT-3.6-120(3,78 MW, 120,0 m)",80,"54° 30′ 0″ N, 7° 48′ 0″ O",in Betrieb,2015,[6][7]
5,Borkum Riffgrund 1,312,"Siemens SWT-4.0-120(4,0 MW, 120,0 m)",78,"53° 58′ 0″ N, 6° 33′ 0″ O",in Betrieb,2015,[8]
6,Butendiek,288,"Siemens SWT-3.6-120(3,6 MW, 120,0 m)",80,"54° 54′ 0″ N, 7° 45′ 0″ O",in Betrieb,2015,[9]
7,DanTysk,"302,4","Siemens SWT-3.6-120(3,78 MW, 120,0 m)",80,"55° 9′ 0″ N, 7° 10′ 30″ O",in Betrieb,2015,[10]
8,Global Tech I,400,"AREVA Multibrid M5000-116(5,0 MW, 116,0 m)",80,"54° 30′ 0″ N, 6° 21′ 30″ O",in Betrieb,2015,[11]
9,MeerwindSüd/Ost,288,"Siemens SWT-3.6-120(3,6 MW, 120,0 m)",80,"54° 23′ 0″ N, 7° 41′ 0″ O",in Betrieb,2015,[12]


In [155]:
#link = "https://de.wikipedia.org/wiki/Liste_der_deutschen_Offshore-Windparks"

#offshores = pd.read_html(link,header=0)[0]






In [156]:
#The coorinates ('Kkordinaten') will be split for two variablles
#Latitude and Longitude

# Function to safely split the coordinates into latitude and longitude
fn = lambda x: pd.Series([i.strip() for i in x.split(',')] if isinstance(x, str) else [None, None])

# Apply the function to split the 'Koordinaten' column into two new columns
coord = offshores['Koordinaten'].apply(fn)

# Rename the resulting columns
coord.rename(columns={0: 'Latitude', 1: 'Longitude'}, inplace=True)

# Add the Latitude and Longitude columns to the original dataframe
offshores['Latitude'] = coord['Latitude']
offshores['Longitude'] = coord['Longitude']



offshores.info()
#we see the presence some NaNs 
#it is because the Windparks are 'In Planung'
#One of the possibilities is to remove them from dataset,
#Second is to label them as well (this another idea ) and 
#to label them in differnet colours on the map 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 10 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Name                                             47 non-null     object
 1   Leistung(MW)                                     45 non-null     object
 2   Windkraftanlagentyp(Leistung, Rotordurchmesser)  45 non-null     object
 3   AnzahlWKAs                                       45 non-null     object
 4   Koordinaten                                      45 non-null     object
 5   Status                                           45 non-null     object
 6   Inbetrieb­nahme(Jahr)                            45 non-null     object
 7   Quellen, Anmerkungen                             45 non-null     object
 8   Latitude                                         45 non-null     object
 9   Longitude                                    

In [157]:
#I removed them so far 
#As well as the rows woth Headers: Nordesee and Ostsee
#offshores.drop(offshores.loc[offshores['Status']=='in Planung'].index, inplace=True)
offshores.drop(offshores.loc[offshores['Name']=='Nordsee'].index, inplace=True)
offshores.drop(offshores.loc[offshores['Name']=='Ostsee'].index, inplace=True)
offshores.drop(offshores.loc[offshores['Status']=='Planung eingestellt'].index, inplace=True)

In [158]:
offshores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 1 to 45
Data columns (total 10 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Name                                             44 non-null     object
 1   Leistung(MW)                                     44 non-null     object
 2   Windkraftanlagentyp(Leistung, Rotordurchmesser)  44 non-null     object
 3   AnzahlWKAs                                       44 non-null     object
 4   Koordinaten                                      44 non-null     object
 5   Status                                           44 non-null     object
 6   Inbetrieb­nahme(Jahr)                            44 non-null     object
 7   Quellen, Anmerkungen                             44 non-null     object
 8   Latitude                                         44 non-null     object
 9   Longitude                                        4

In [159]:
offshores

,Name,Leistung(MW),"Windkraftanlagentyp(Leistung, Rotordurchmesser)",AnzahlWKAs,Koordinaten,Status,Inbetrieb­nahme(Jahr),"Quellen, Anmerkungen",Latitude,Longitude
1,alpha ventus,"60,48","6 × REpower 5M(5,08 MW, 126,5 m)6 ×AREVAM5000-...",12,"54° 0′ 30″ N, 6° 35′ 54″ O",in Betrieb,2010,,54° 0′ 30″ N,6° 35′ 54″ O
2,BARD Offshore I,400,"BARD5.0(5,0 MW, 122,0 m)",80,"54° 21′ 30″ N, 5° 58′ 30″ O",in Betrieb,2013,,54° 21′ 30″ N,5° 58′ 30″ O
3,Riffgat,"113,4","Siemens SWT-3.6-120(3,78 MW, 120,0 m)",30,"53° 41′ 24″ N, 6° 28′ 48″ O",in Betrieb,2014,[5],53° 41′ 24″ N,6° 28′ 48″ O
4,Amrumbank West,"302,4","Siemens SWT-3.6-120(3,78 MW, 120,0 m)",80,"54° 30′ 0″ N, 7° 48′ 0″ O",in Betrieb,2015,[6][7],54° 30′ 0″ N,7° 48′ 0″ O
5,Borkum Riffgrund 1,312,"Siemens SWT-4.0-120(4,0 MW, 120,0 m)",78,"53° 58′ 0″ N, 6° 33′ 0″ O",in Betrieb,2015,[8],53° 58′ 0″ N,6° 33′ 0″ O
6,Butendiek,288,"Siemens SWT-3.6-120(3,6 MW, 120,0 m)",80,"54° 54′ 0″ N, 7° 45′ 0″ O",in Betrieb,2015,[9],54° 54′ 0″ N,7° 45′ 0″ O
7,DanTysk,"302,4","Siemens SWT-3.6-120(3,78 MW, 120,0 m)",80,"55° 9′ 0″ N, 7° 10′ 30″ O",in Betrieb,2015,[10],55° 9′ 0″ N,7° 10′ 30″ O
8,Global Tech I,400,"AREVA Multibrid M5000-116(5,0 MW, 116,0 m)",80,"54° 30′ 0″ N, 6° 21′ 30″ O",in Betrieb,2015,[11],54° 30′ 0″ N,6° 21′ 30″ O
9,MeerwindSüd/Ost,288,"Siemens SWT-3.6-120(3,6 MW, 120,0 m)",80,"54° 23′ 0″ N, 7° 41′ 0″ O",in Betrieb,2015,[12],54° 23′ 0″ N,7° 41′ 0″ O
10,Nordsee Ost,"295,2","Senvion6.2M126(6,15 MW, 126,0 m)",48,"54° 26′ 0″ N, 7° 41′ 0″ O",in Betrieb,2015,,54° 26′ 0″ N,7° 41′ 0″ O


In [160]:
Latitude_cor = []

# Iterate over each latitude in the 'Latitude' column
for lat in offshores['Latitude']:
    if pd.notna(lat):  # Check if the value is not None or NaN
        # Split the latitude by the degree, minute, and second symbols
        deg, minutes, seconds = re.split('[°\′″]', lat)
        Latitude_cor.append(float(deg) + float(minutes)/60 + float(seconds)/(60*60))
    else:
        Latitude_cor.append(np.nan)  # Append NaN if the value is None or NaN

Longitude_cor = []

# Iterate over each longitude in the 'Longitude' column
for long in offshores['Longitude']:
    if pd.notna(long):  # Check if the value is not None or NaN
        # Split the longitude by the degree, minute, and second symbols
        deg, minutes, seconds = re.split('[°\′″]', long)
        Longitude_cor.append(float(deg) + float(minutes)/60 + float(seconds)/(60*60))
    else:
        Longitude_cor.append(np.nan)  # Append NaN if the value is None or NaN

# Check the length of the resulting lists
print(len(Latitude_cor))
print(len(Longitude_cor))

ValueError: too many values to unpack (expected 3)

In [161]:
offshores['Latitude'] = offshores['Latitude'].str.split('N', n=1).str.get(0)
offshores['Longitude'] = offshores['Longitude'].str.split('O', n=1).str.get(0)


offshores['Latitude'] = offshores['Latitude'].str.split('″', n=1).str.get(0)
offshores['Longitude'] = offshores['Longitude'].str.split('″', n=1).str.get(0)


Latitude_cor=[]

for lat in offshores['Latitude']:
    deg, minutes, seconds =  re.split('[°\′]', lat)
    Latitude_cor.append(float(deg) + float(minutes)/60 + float(seconds)/(60*60))
    
Longitude_cor=[]

for long in offshores['Longitude']:
    deg, minutes, seconds =  re.split('[°\′]', long)
    Longitude_cor.append(float(deg) + float(minutes)/60 + float(seconds)/(60*60))
    
len(Latitude_cor) 

44

In [162]:
offshores['Latitude_cor']=Latitude_cor
offshores['Longitude_cor']=Longitude_cor
offshores.head(3)

,Name,Leistung(MW),"Windkraftanlagentyp(Leistung, Rotordurchmesser)",AnzahlWKAs,Koordinaten,Status,Inbetrieb­nahme(Jahr),"Quellen, Anmerkungen",Latitude,Longitude,Latitude_cor,Longitude_cor
1,alpha ventus,"60,48","6 × REpower 5M(5,08 MW, 126,5 m)6 ×AREVAM5000-...",12,"54° 0′ 30″ N, 6° 35′ 54″ O",in Betrieb,2010,,54° 0′ 30,6° 35′ 54,54.008333,6.598333
2,BARD Offshore I,400,"BARD5.0(5,0 MW, 122,0 m)",80,"54° 21′ 30″ N, 5° 58′ 30″ O",in Betrieb,2013,,54° 21′ 30,5° 58′ 30,54.358333,5.975000
3,Riffgat,"113,4","Siemens SWT-3.6-120(3,78 MW, 120,0 m)",30,"53° 41′ 24″ N, 6° 28′ 48″ O",in Betrieb,2014,[5],53° 41′ 24,6° 28′ 48,53.690000,6.480000


In [163]:
offshores=offshores[['Name', 'AnzahlWKAs', 'Leistung(MW)', 'Latitude_cor', 'Longitude_cor','Status', 'Windkraftanlagentyp(Leistung, Rotordurchmesser)', 'Inbetrieb­nahme(Jahr)']]
offshores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 1 to 45
Data columns (total 8 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Name                                             44 non-null     object 
 1   AnzahlWKAs                                       44 non-null     object 
 2   Leistung(MW)                                     44 non-null     object 
 3   Latitude_cor                                     44 non-null     float64
 4   Longitude_cor                                    44 non-null     float64
 5   Status                                           44 non-null     object 
 6   Windkraftanlagentyp(Leistung, Rotordurchmesser)  44 non-null     object 
 7   Inbetrieb­nahme(Jahr)                            44 non-null     object 
dtypes: float64(2), object(6)
memory usage: 3.1+ KB


In [164]:
# Replace commas with dots and convert to float
offshores['Leistung(MW)'] = offshores['Leistung(MW)'].str.replace(',', '.').astype(float)

# Now, the 'Leistung' column should be a float type and ready for calculations
print(offshores['Leistung(MW)'])

1       60.48
2      400.00
3      113.40
4      302.40
5      312.00
6      288.00
7      302.40
8      400.00
9      288.00
10     295.20
11     200.00
12     346.00
13     264.00
14     110.70
15     332.10
16     288.00
17     402.00
18     464.80
19     260.40
20     396.00
21     521.85
22     117.60
23     203.20
24     342.00
25     242.00
26     913.00
27     960.00
28     660.00
29     980.00
30     900.00
31     630.00
32     270.00
33    2000.00
34    2000.00
35    2000.00
37      48.30
38     288.00
39     353.50
40     384.00
41     257.20
42     494.00
43     927.00
44     300.00
45    1000.00
Name: Leistung(MW), dtype: float64


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_987/2984299201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offshores['Leistung(MW)'] = offshores['Leistung(MW)'].str.replace(',', '.').astype(float)


In [165]:
offshores['Windkraftanlagentyp(Leistung, Rotordurchmesser)'].unique()

array(['6 × REpower 5M(5,08\xa0MW, 126,5\xa0m)6 ×AREVAM5000-116(5,0\xa0MW, 116,0\xa0m)',
       'BARD5.0(5,0\xa0MW, 122,0\xa0m)',
       'Siemens SWT-3.6-120(3,78\xa0MW, 120,0\xa0m)',
       'Siemens SWT-4.0-120(4,0\xa0MW, 120,0\xa0m)',
       'Siemens SWT-3.6-120(3,6\xa0MW, 120,0\xa0m)',
       'AREVA Multibrid M5000-116(5,0\xa0MW, 116,0\xa0m)',
       'Senvion6.2M126(6,15\xa0MW, 126,0\xa0m)',
       'Siemens SWT-6.0-154(6,3\xa0MW, 154,0\xa0m)',
       'Senvion 6.2M126(6,15\xa0MW, 126,0\xa0m)',
       'Siemens SWT-4.0-130(4,0\xa0MW, 130,0\xa0m)',
       'Siemens SWT-6.0-154(6,0\xa0MW, 154,0\xa0m)',
       'MHI Vestas V164-8.0 MW(8,3\xa0MW, 164,0\xa0m)',
       'MHI Vestas V164-8.0 MW(8,4\xa0MW, 164,0\xa0m)',
       'GE Haliade 150-6 MW(6,0\xa0MW, 151,0\xa0m)',
       'Siemens SWT-7.0-154(7,35\xa0MW, 154,0\xa0m)',
       'Senvion 6.3M152(6,33\xa0MW, 152,0\xa0m)',
       'Siemens Gamesa SG 8.0-167 DD(9,0\xa0MW, 167,0\xa0m)',
       'Siemens Gamesa SG 11.0-200 DD(11,0\xa0MW, 200,0\xa0m)'

In [166]:
# Replace non-breaking spaces (\xa0) with regular spaces
offshores['Windkraftanlagentyp(Leistung, Rotordurchmesser)'] = offshores['Windkraftanlagentyp(Leistung, Rotordurchmesser)'].str.replace('\xa0', ' ', regex=False)

# You can also replace them with an empty string if you don't want any spaces
# offshores['Leistung(MW)'] = offshores['Leistung(MW)'].str.replace('\xa0', '', regex=False)

# Check the cleaned data
print(offshores['Windkraftanlagentyp(Leistung, Rotordurchmesser)'])

1     6 × REpower 5M(5,08 MW, 126,5 m)6 ×AREVAM5000-...
2                              BARD5.0(5,0 MW, 122,0 m)
3                 Siemens SWT-3.6-120(3,78 MW, 120,0 m)
4                 Siemens SWT-3.6-120(3,78 MW, 120,0 m)
5                  Siemens SWT-4.0-120(4,0 MW, 120,0 m)
6                  Siemens SWT-3.6-120(3,6 MW, 120,0 m)
7                 Siemens SWT-3.6-120(3,78 MW, 120,0 m)
8            AREVA Multibrid M5000-116(5,0 MW, 116,0 m)
9                  Siemens SWT-3.6-120(3,6 MW, 120,0 m)
10                     Senvion6.2M126(6,15 MW, 126,0 m)
11           AREVA Multibrid M5000-116(5,0 MW, 116,0 m)
12                 Siemens SWT-6.0-154(6,3 MW, 154,0 m)
13                 Siemens SWT-6.0-154(6,3 MW, 154,0 m)
14                    Senvion 6.2M126(6,15 MW, 126,0 m)
15                    Senvion 6.2M126(6,15 MW, 126,0 m)
16                 Siemens SWT-4.0-130(4,0 MW, 130,0 m)
17                 Siemens SWT-6.0-154(6,0 MW, 154,0 m)
18              MHI Vestas V164-8.0 MW(8,3 MW, 1

/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_987/281557747.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offshores['Windkraftanlagentyp(Leistung, Rotordurchmesser)'] = offshores['Windkraftanlagentyp(Leistung, Rotordurchmesser)'].str.replace('\xa0', ' ', regex=False)


In [167]:
offshores.to_csv('Offshores_Windparks.csv', index=False)

In [168]:
df=pd.read_csv('Offshores_Windparks.csv')

In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 8 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Name                                             44 non-null     object 
 1   AnzahlWKAs                                       36 non-null     float64
 2   Leistung(MW)                                     44 non-null     float64
 3   Latitude_cor                                     44 non-null     float64
 4   Longitude_cor                                    44 non-null     float64
 5   Status                                           44 non-null     object 
 6   Windkraftanlagentyp(Leistung, Rotordurchmesser)  37 non-null     object 
 7   Inbetrieb­nahme(Jahr)                            44 non-null     object 
dtypes: float64(4), object(4)
memory usage: 2.9+ KB


In [151]:
df

,Name,AnzahlWKAs,Leistung(MW),Latitude_cor,Longitude_cor,Status,"Windkraftanlagentyp(Leistung, Rotordurchmesser)",Inbetrieb­nahme(Jahr)
0,alpha ventus,12.0,60.48,54.008333,6.598333,in Betrieb,"6 × REpower 5M(5,08 MW, 126,5 m)6 ×AREVAM5000-...",2010
1,BARD Offshore I,80.0,400.00,54.358333,5.975000,in Betrieb,"BARD5.0(5,0 MW, 122,0 m)",2013
2,Riffgat,30.0,113.40,53.690000,6.480000,in Betrieb,"Siemens SWT-3.6-120(3,78 MW, 120,0 m)",2014
3,Amrumbank West,80.0,302.40,54.500000,7.800000,in Betrieb,"Siemens SWT-3.6-120(3,78 MW, 120,0 m)",2015
4,Borkum Riffgrund 1,78.0,312.00,53.966667,6.550000,in Betrieb,"Siemens SWT-4.0-120(4,0 MW, 120,0 m)",2015
5,Butendiek,80.0,288.00,54.900000,7.750000,in Betrieb,"Siemens SWT-3.6-120(3,6 MW, 120,0 m)",2015
6,DanTysk,80.0,302.40,55.150000,7.175000,in Betrieb,"Siemens SWT-3.6-120(3,78 MW, 120,0 m)",2015
7,Global Tech I,80.0,400.00,54.500000,6.358333,in Betrieb,"AREVA Multibrid M5000-116(5,0 MW, 116,0 m)",2015
8,MeerwindSüd/Ost,80.0,288.00,54.383333,7.683333,in Betrieb,"Siemens SWT-3.6-120(3,6 MW, 120,0 m)",2015
9,Nordsee Ost,48.0,295.20,54.433333,7.683333,in Betrieb,"Senvion6.2M126(6,15 MW, 126,0 m)",2015
